In [1]:
# from autogenstudio.src.autogenstudio.workflowmanager import WorkflowManager

In [2]:
import logging
from autogen_agentchat.logging import ConsoleLogHandler
from autogen_agentchat import EVENT_LOGGER_NAME
logger = logging.getLogger(EVENT_LOGGER_NAME)
logger.addHandler(ConsoleLogHandler())
logger.setLevel(logging.INFO)


In [3]:
from autogen_agentchat.agents import  AssistantAgent
from autogen_agentchat.task import MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_core.components.tools import FunctionTool
from autogen_ext.models import OpenAIChatCompletionClient



# define a tool
async def get_weather(city: str) -> str:
    return f"The weather in {city} is 73 degrees and Sunny."


# wrap the tool for use with the agent
get_weather_tool = FunctionTool(get_weather, description="Get the weather for a city")

# define an agent
weather_agent = AssistantAgent(
    name="writing_agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-2024-08-06"),
    tools=[get_weather_tool],
)

# add the agent to a team
agent_team = RoundRobinGroupChat([weather_agent])
# Note: if running in a Python file directly you'll need to use asyncio.run(agent_team.run(...)) instead of await agent_team.run(...)
result = await agent_team.run(
    task="What is the weather in New York?",
    termination_condition=MaxMessageTermination(max_messages=2),
)
print("\n", result)


--------------------------------------------------------------------------- 
[2024-10-28T15:11:19.981976]:

What is the weather in New York?
--------------------------------------------------------------------------- 
[2024-10-28T15:11:21.470832], writing_agent:

The weather in New York is 73 degrees and sunny.
--------------------------------------------------------------------------- 
[2024-10-28T15:11:21.471168], Termination:

Maximal number of messages 2 reached, current message count: 2
 TaskResult(messages=[TextMessage(source='user', content='What is the weather in New York?'), TextMessage(source='writing_agent', content='The weather in New York is 73 degrees and sunny.')])


In [5]:
result

TaskResult(messages=[TextMessage(source='user', content='What is the weather in New York?'), TextMessage(source='writing_agent', content='The weather in New York is 73 degrees and sunny.')])

In [9]:

from typing import Any, List, Literal, Optional
from pydantic.dataclasses import dataclass
from autogen_agentchat.agents import AssistantAgent, CodingAssistantAgent
from autogen_agentchat.task import MaxMessageTermination, StopMessageTermination, TextMentionTermination
from autogen_ext.models import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat

AgentTypes = AssistantAgent | CodingAssistantAgent
TeamTypes = RoundRobinGroupChat | SelectorGroupChat
ModelTypes = OpenAIChatCompletionClient 
TerminationTypes = MaxMessageTermination | StopMessageTermination | TextMentionTermination

@dataclass
class ModelConfig:
    model: str
    model_type: Literal["OpenAIChatCompletionClient"]

@dataclass
class ToolConfig:
    name: str
    description: str
    content: str 

@dataclass
class AgentConfig:
    name: str 
    agent_type: Literal["AssistantAgent", "CodingAssistantAgent"]
    system_message: Optional[str] = None
    model_client: Optional[ModelConfig] = None 
    tools:Optional[List[ToolConfig]] = None
    description: Optional[str] = None  

@dataclass
class TerminationConfig: 
    termination_type: Literal["MaxMessageTermination", "StopMessageTermination", "TextMentionTermination"]
    max_messages: Optional[int] = None
    text: Optional[str] = None

@dataclass
class TeamConfig:
    name: str
    participants: List[AgentConfig]
    termination_condition:  TerminationConfig
    team_type: Literal["RoundRobinGroupChat", "SelectorGroupChat"]
    model_client: Optional[ModelConfig] = None

class Provider():
    def __init__(self):
        pass

    def load_model(self, model_config: ModelConfig | dict) -> ModelTypes:
        if isinstance(model_config, dict):
            try:
                model_config = ModelConfig(**model_config)
            except:
                raise ValueError("Invalid model config")
        model = None
        if model_config.model_type == "OpenAIChatCompletionClient":
            model = OpenAIChatCompletionClient(model=model_config.model)
        return model
    
    def _func_from_string(self, content: str) -> Any:
        """
        Convert a string containing function code into a callable function object.
        
        Args:
            content (str): String containing the function code, with proper indentation
            
        Returns:
            Callable: The compiled function object
        """
        # Create a namespace for the function
        namespace = {}
        
        # Ensure content is properly dedented if it contains indentation
        lines = content.split('\n')
        if len(lines) > 1:
            # Find the minimum indentation (excluding empty lines)
            indents = [len(line) - len(line.lstrip()) for line in lines if line.strip()]
            min_indent = min(indents) if indents else 0
            # Remove the minimum indentation from each line
            lines = [line[min_indent:] if line.strip() else line for line in lines]
            content = '\n'.join(lines)
        
        try:
            # Execute the function definition in the namespace
            exec(content, namespace)
            
            # Find and return the function object
            # Get the first callable object from the namespace
            for item in namespace.values():
                if callable(item) and not isinstance(item, type):
                    return item
                    
            raise ValueError("No function found in the provided code")
        except Exception as e:
            raise ValueError(f"Failed to create function from string: {str(e)}")
    
    def load_tool(self, tool_config: ToolConfig | dict) -> FunctionTool:
        if isinstance(tool_config, dict):
            try:
                tool_config = ToolConfig(**tool_config)
            except:
                raise ValueError("Invalid tool config")
        tool = FunctionTool(name=tool_config.name, description=tool_config.description, func=self._func_from_string(tool_config.content))
        return tool

    def load_agent(self, agent_config: AgentConfig | dict) -> AgentTypes:
        if isinstance(agent_config, dict):
            try:
                agent_config = AgentConfig(**agent_config)
            except:
                raise ValueError("Invalid agent config")
        agent = None
        if agent_config.agent_type == "AssistantAgent":
            model_client =  self.load_model(agent_config.model_client)
            system_message = agent_config.system_message if agent_config.system_message else "You are a helpful AI assistant. Solve tasks using your tools. Reply with 'TERMINATE' when the task has been completed."
            tools = [self.load_tool(tool) for tool in agent_config.tools]
            agent = AssistantAgent(name=agent_config.name, model_client=model_client, tools=tools, system_message=system_message)
        
        return agent
    
    def load_termination(self, termination_config: TerminationConfig | dict) -> TerminationTypes: 
        if isinstance(termination_config, dict):
            try:
                termination_config = TerminationConfig(**termination_config)
            except:
                raise ValueError("Invalid termination config")
        termination = None
        if termination_config.termination_type == "MaxMessageTermination":
            termination = MaxMessageTermination(max_messages=termination_config.max_messages)
        elif termination_config.termination_type == "StopMessageTermination":
            termination = StopMessageTermination()
        elif termination_config.termination_type == "TextMentionTermination":
            termination = TextMentionTermination(text=termination_config.text)
        return termination
    
    def load_team(self, team_config: TeamConfig | dict, team_type: TeamTypes) -> TeamTypes:
        if isinstance(team_config, dict):
            try:
                team_config = TeamConfig(**team_config)
            except:
                raise ValueError("Invalid team config")
        team = None
        agents = []
        # tbd on termination condition
        for agent_config in team_config.participants:
            agent = self.load_agent(agent_config)
            agents.append(agent)
        if team_type == RoundRobinGroupChat:
            team = RoundRobinGroupChat(agents )
        elif team_type == SelectorGroupChat:
            team = SelectorGroupChat(agents)

        return team

In [10]:
model_json_spec = {
    "model": "gpt-4o-2024-08-06",
    "model_type": "OpenAIChatCompletionClient"
}

tool_json_spec = {
    "name": "get_weather",
    "description": "Get the weather for a city",
    "content": "async def get_weather(city: str) -> str:\n    return f\"The weather in {city} is 73 degrees and Sunny.\""
}

agent_json_spec = {
    "name": "writing_agent",
    "model_client":  model_json_spec,
    "tools": [
        tool_json_spec
    ],
    "agent_type": "AssistantAgent"
}

termination_json_spec = { 
    "termination_type": "StopMessageTermination"
}

team_json_spec = { 
    "name": "weather_team",
    "participants": [agent_json_spec],
    "termination_condition": termination_json_spec,
    "team_type": "RoundRobinGroupChat",
    "model_client": None, 
}

provider = Provider() 

agent = provider.load_agent(agent_json_spec)
termination = provider.load_termination(termination_json_spec)
team = provider.load_team(team_json_spec, RoundRobinGroupChat)


In [11]:
result = await team.run(task="What is the weather in New York?", termination_condition=termination)


--------------------------------------------------------------------------- 
[2024-10-28T15:13:47.853189]:

What is the weather in New York?
--------------------------------------------------------------------------- 
[2024-10-28T15:13:49.107659], writing_agent:

The weather in New York is currently 73 degrees and sunny.
--------------------------------------------------------------------------- 
[2024-10-28T15:13:49.493807], writing_agent:

TERMINATE
--------------------------------------------------------------------------- 
[2024-10-28T15:13:49.494198], Termination:

Stop message received